<a href="https://colab.research.google.com/github/benliebersohn/alg-ds-lab2/blob/master/PythonForGIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Some useful and practical python skills/topics

- [Using `censusdata` library to pull ACS data](#census)
- [Geocoding using `geopandas`](#geocoding)
- [Spatial join using `geopandas` (e.g. point-in-polygon)](#sj)
- [Interactive map with `bokeh`](#bokeh)



Further resources:

`censusdata`: https://jtleider.github.io/censusdata/

`geopandas` geocoding: https://automating-gis-processes.github.io/CSC18/lessons/L3/geocoding.html

`geopandas` spatial join: https://geopandas.org/mergingdata.html

`bokeh` example: https://docs.bokeh.org/en/latest/docs/gallery/texas.html

In [1]:
# Use the hastag/point sign/sharp symbol '#' to make a comment
# A comment in the code will be ignored
#print("This won't appear - commented out")
print("This will appear - no comment")

This will appear - no comment


In [2]:
!unzip cb_2018_us_county_20m.zip
# This extracts our compressed folder. If it "cannot find or open" then you need to upload the zip file
# Notice all the output, this is useful for understanding what is happening

unzip:  cannot find or open cb_2018_us_county_20m.zip, cb_2018_us_county_20m.zip.zip or cb_2018_us_county_20m.zip.ZIP.


In [3]:
!apt-get install -y python-rtree; #Semicolon at the end suppresses the output when installing the rtree library

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python-rtree


In [4]:
!pip install pygeos #We install some libraries with the tool called "pip"
!pip install geopandas; #semicolon suppressed output
# If you want to learn more about these libraries, Google search "Pygeos" or "geopandas"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.0 MB/s eta 0:00:00


In [5]:
import pygeos # Now that we installed pygeos, we still need to import them
import pandas as pd # by using the shorter names we save some time later on
import numpy as np # instead of calling it numpy we call it np
import geopandas as gpd # even though we are calling it gpd it is still widely known as "geopandas"

/usr/local/lib/python3.10/dist-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-5-14c62f28b034>:4: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling Py

<a id='census'></a>
### Census API


In [6]:
!pip install censusdata # We install and import the censusdata library
import censusdata # This library loads census data for us, so we don't need to go to the US Census Bureau website

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 58.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for censusdata: filename=CensusData-1.15.post1-py3-none-any.whl size=28205746 sha256=f25b1e0b8c08e585e040f3207ba9b70669bd94d3b99e33c09ab0f5bfefbf6274
  Stored in directory: /root/.cache/pip/wheels/40/0a/09/c996fa9cc686a1efb90426ce5fbaac1e2e0d7e0efbb3939a85
Successfully built censusdata


In [7]:
#Pull Data for all US counties
#The code associated with attributes is found using printtable below

age_race = censusdata.download('acs5' #Dataset to use
                               , 2018, # year 2014-2018, the most recent one
                               censusdata.censusgeo([('county',"*")]), #Geography, * = wildcard for all
                                   ['DP05_0001E', #Total Pop
                                    'DP05_0021PE', #Pct_65+
                                    'DP05_0038PE', #Pct_black
                                    'DP05_0071PE', #Pct_hispanic
                                    'DP05_0002E', #Male
                                    'DP05_0003E', #Female
                                   ],
                                   tabletype='profile')
#Re-name the column names
age_race.columns = ['total_pop','pct_65_over','pct_black','pct_hispanic','males','females']

In [8]:
data.head()
#age_race #try uncommenting this line and commenting out the above line. This will return different things

NameError: ignored

In [ ]:
#Check each Data Profile
# printtable shows the variable, table, label
censusdata.printtable(censusdata.censustable('acs5', 2015, 'DP03'))

In [ ]:
#Check on state code
censusdata.geographies(censusdata.censusgeo([('state', '*')]), 'acs5', 2018)

In [ ]:
#Pull Data for only Illinois counties
IL_age_race = censusdata.download('acs5', 2018, censusdata.censusgeo([('state', '17'),('county',"*")]),
                                   ['DP05_0001E', #Total Pop
                                    'DP05_0021PE', #Pct_65+
                                    'DP05_0038PE', #Pct_black
                                    'DP05_0071PE', #Pct_hispanic
                                    'DP05_0002E', #Male
                                    'DP05_0003E', #Female
                                   ],
                                   tabletype='profile')
IL_age_race.columns = ['total_pop','pct_65_over','pct_black','pct_hispanic','males','females']

In [ ]:
IL_age_race.head()

In [ ]:
#Concatenate these ACS attributes together
IL_acs_18 = pd.concat([IL_age_race,IL_edu],axis=1)

In [ ]:
IL_acs_18

In [ ]:
#You can export the data to a csv file like this:
censusdata.export.exportcsv("IL_acs_18.csv", IL_acs_18)

In [ ]:
#Let's read it back in
IL_acs_18 = pd.read_csv("IL_acs_18.csv")

In [ ]:
IL_acs_18

In [ ]:
#Here we create a GEOID field, will be used to join with TIGER boundary files
IL_acs_18["GEOID"]= (IL_acs_18['state'].astype(str).str.zfill(2) + IL_acs_18['county'].astype(str).str.zfill(3)).astype(int)
# common key between shp and csv is FIPS code
# county code is 3 digit, so 83 should be 083 which is why we add a 0 to some

In [ ]:
IL_acs_18

In [ ]:
#Read-in the TIGER boundary files
#Downloaded from https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
counties = gpd.read_file("/content/cb_2018_us_county_20m/cb_2018_us_county_20m.shp")

In [ ]:
#This happens elsewhere that we need to make sure the format of the GEOID to be the same.
#Here I create a new attribute called "GEOID_int"

counties["GEOID_int"] = counties.GEOID.astype(int)

In [ ]:
#Table join the boundary file with the acs file
#"Join by Attribute" in ArcGIS
#first data frame is the left, second is on the right, ,we specify the keys and join based on the shape on the left
IL_acs = gpd.GeoDataFrame(pd.merge(IL_acs_18,counties,how="left",left_on="GEOID",right_on="GEOID_int"))

In [ ]:
IL_acs.head()

In [ ]:
IL_acs.plot(column="pct_hispanic",legend=True)


In [ ]:
#exporting to shapefile for more analysis with ArcGIS Pro
IL_acs.to_file("name.shp")

In [ ]:
IL_acs.centroid # Calculate the centroid of each county in the shapefile


In [ ]:
IL_acs.centroid.x.values # convert to numpy array

<a id='bokeh'></a>
### Interactive mapping


In [ ]:
#We use an awesome package called bokeh (I love this package!)

from bokeh.io import output_file, show,output_notebook
from bokeh.models import ColumnDataSource,ColorBar,HoverTool
from bokeh.transform import linear_cmap
from bokeh.plotting import figure
from bokeh.palettes import Spectral6 #https://docs.bokeh.org/en/latest/docs/reference/palettes.html

In [ ]:
#To make your map be outputted inline
output_notebook()

In [ ]:
#Don't change this!!!
#Just copy this whole cell.
#This is a helper function for converting a GeoDataFrame to the format that bokeh can recognize.

def gpd_bokeh(df):
    """Convert geometries from geopandas to bokeh format"""
    nan = float('nan')
    lons = []
    lats = []
    for i,shape in enumerate(df.geometry.values):
        if shape.geom_type == 'MultiPolygon':
            gx = []
            gy = []
            ng = len(shape.geoms) - 1
            for j,member in enumerate(shape.geoms):
                xy = np.array(list(member.exterior.coords))
                xs = xy[:,0].tolist()
                ys = xy[:,1].tolist()
                gx.extend(xs)
                gy.extend(ys)
                if j < ng:
                    gx.append(nan)
                    gy.append(nan)
            lons.append(gx)
            lats.append(gy)

        else:
            xy = np.array(list(shape.exterior.coords))
            xs = xy[:,0].tolist()
            ys = xy[:,1].tolist()
            lons.append(xs)
            lats.append(ys)

    return lons,lats

In [ ]:
#IL_acs
IL_acs.plot()

In [ ]:
#Feed in the data for bokeh

lons, lats = gpd_bokeh(IL_acs)

source = ColumnDataSource(data=dict( #specify the x, y coordinates, and the data we want to put in to the map
        x=lons,
        y=lats,
        name = IL_acs['NAME_x'], #Add any columns you want to bokeh. NAME_x is the county name
        population = IL_acs['total_pop'],
        Pct_Bach = IL_acs['pct_bach'])) #pct_bach is the percentage of residents with a bachelors degree

In [ ]:
#Create a color map
color_mapper = linear_cmap(field_name='Pct_Bach', #the field to map
                           palette=Spectral6, #the color to use
                           low=min(IL_acs['pct_bach']) , # The low and high bounds for your color map
                           high=max(IL_acs['pct_bach']))


In [ ]:
#Add tools you want
TOOLS = "pan,wheel_zoom,reset,hover,save"

In [ ]:
#Create a plot frame with size and title
map = figure(plot_width=450, plot_height=600,title="Education Attainment of Illinois", tools=TOOLS)

#Add the polygon patches
map.patches('x', 'y', source=source, line_color="white", line_width=0.1, color=color_mapper)

#Add the hover tool and the hover field to display
map.select_one(HoverTool).tooltips = [
    ('County Name', '@name'), #each tuple needs to follow this format.
    ('Population', '@population'),
    ('% of People with a Bachelors degree', '@Pct_Bach')
]

#Add your colorbar
color_bar = ColorBar(color_mapper=color_mapper['transform'], width=16, location=(0,0))
map.add_layout(color_bar, 'right')

In [ ]:
#Show the map
show(map)

In [ ]:
#You can export your map to a html file.
output_file("Edu_IL.html")